In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [9, 8]

import cv2
import numpy as np
import pandas as pd
pd.set_option('display.width', 74)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 20)

In [2]:
import os
import datetime
import itertools

from sklearn.metrics import confusion_matrix

DATADIR = "data/keras"
NUM_CHARACTERS = len(os.listdir(os.path.join(DATADIR, "train", "images")))
BATCH_SIZE = 16
LEARNING_RATE = 0.01

IMAGE_ROW_SIZE = 584
IMAGE_COLUMN_SIZE = 480

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input

def make_generator(folder="train",
                   data_gen_args={"fill_mode": "constant",
                                  "cval": 0,
                                  "width_shift_range": 0.05,
                                  "height_shift_range": 0.05,
                                  "zoom_range": 0.1,
                                  "horizontal_flip": True,
                                  "rescale": 1.0 / 255,
                                  "preprocessing_function": preprocess_input},
                   data_flow_args={"seed": 1,
                                   "batch_size": BATCH_SIZE}):

    image_datagen = ImageDataGenerator(**data_gen_args)

    image_generator = image_datagen.flow_from_directory(
        directory=os.path.join(DATADIR, folder, "images"),
        target_size=(IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE),
        color_mode='rgb',
        **data_flow_args)

    return image_generator


def count_images(folder="train"):
    image_directory = os.path.join(DATADIR, folder, "images")
    data_size = 0

    for char_name in os.listdir(image_directory):
        char_directory = os.path.join(image_directory, char_name)
        data_size += len(os.listdir(char_directory))

    return data_size


def steps_per_epoch(folder="train", batch_size=BATCH_SIZE):
    return count_images(folder) // batch_size

Using TensorFlow backend.


In [6]:
# ugh this is garbage

from keras.applications.vgg16 import VGG16

model = VGG16(include_top=False, weights="imagenet")

image_count_train = count_images("train")
generator_train = make_generator("train",
                                 data_gen_args={"preprocessing_function": preprocess_input},
                                 data_flow_args={"batch_size": 1,
                                                 "class_mode": None,
                                                 "shuffle": False})
generator_train_labels = make_generator("train",
                                        data_gen_args={"preprocessing_function": preprocess_input},
                                        data_flow_args={"batch_size": 1,
                                                        "shuffle": False})
labels_train = []
for i, (image, labels) in enumerate(generator_train_labels):
    if i == image_count_train:
        break
    else:
        labels_train.append(labels[0])
labels_train = np.array(labels_train)

bottleneck_features_train = model.predict_generator(generator_train,
                                                    image_count_train)
with open(os.path.join(DATA_DIR, 'bottleneck_features_train.npy'), 'w') as train:
    np.save(train, bottleneck_features_train)
with open(os.path.join(DATA_DIR, 'bottleneck_feature_labels_train.npy'), 'w') as train:
    np.save(train, labels_train)

image_count_valid = count_images("valid")
generator_valid = make_generator("valid",
                                 data_gen_args={"rescale": 1.0 / 255},
                                 data_flow_args={"batch_size": 1,
                                                 "class_mode": None,
                                                 "shuffle": False})

generator_valid_labels = make_generator("valid",
                                        data_gen_args={"preprocessing_function": preprocess_input},
                                        data_flow_args={"batch_size": 1,
                                                        "shuffle": False})
labels_valid = []
for i, (image, labels) in enumerate(generator_valid_labels):
    if i == image_count_valid:
        break
    else:
        labels_valid.append(labels[0])
labels_valid = np.array(labels_valid)

bottleneck_features_valid = model.predict_generator(generator_valid,
                                                    image_count_valid)
with open(os.path.join(DATA_DIR, 'bottleneck_features_valid.npy'), 'w') as valid:
    np.save(valid, bottleneck_features_valid)

with open(os.path.join(DATA_DIR, 'bottleneck_feature_labels_valid.npy'), 'w') as valid:
    np.save(valid, labels_valid)

58892288/58889256 [==============================] - 1s 0us/step
Found 13383 images belonging to 8 classes.


KeyboardInterrupt: 

In [ ]:
from keras.optimizers import Adam

def get_labels(folder="train"):
    image_directory = os.path.join(DATADIR, folder, "images")
    char_list = os.listdir(image_directory)
    labels = []

    for i, char_name in enumerate(char_list):
        one_hot = [0] * len(char_list)
        one_hot[i] = 1
        char_directory = os.path.join(image_directory, char_name)
        data_size = len(os.listdir(char_directory))
        labels.extend([one_hot for elt in range(data_size)])

    return labels

train_labels = np.array(get_labels("train"))
with open(os.path.join(DATA_DIR, 'bottleneck_features_train.npy'), 'r') as train:
    train_data = np.load(train)

valid_labels = np.array(get_labels("valid"))
with open(os.path.join(DATA_DIR, 'bottleneck_features_valid.npy'), 'r') as valid:
    valid_data = np.load(valid)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu', kernel_initializer='lecun_uniform'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CHARACTERS, activation='softmax'))

model.compile(optimizer=Adam(lr=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=BATCH_SIZE,
          validation_data=(valid_data, valid_labels))

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Input, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam

def init_model(target_num=4, dropout_ratio=0.5, learning_rate=0.0001):
    input_shape = (IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE, 3)

    # Fine-tune prediction layer
    pretrained_model = VGG16(include_top=False, weights='imagenet',
                             input_shape=input_shape)
    for layer in pretrained_model.layers:
        layer.trainable = False

    output_tensor = pretrained_model.output
    output_tensor = Flatten()(output_tensor)
    output_tensor = Dense(128, activation='relu', 
                          kernel_initializer='lecun_uniform')(output_tensor)
    # output_tensor = Dense(128, activation='relu', 
    #                       kernel_initializer='lecun_uniform')(output_tensor)
    output_tensor = Dense(target_num, activation="softmax", 
                          kernel_initializer='lecun_uniform', 
                          name="predictions")(output_tensor)

    # Define and compile the model
    model = Model(inputs=pretrained_model.input, outputs=output_tensor)
    model.compile(optimizer=Adam(lr=learning_rate),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [ ]:
model = init_model(target_num=NUM_CHARACTERS, learning_rate=LEARNING_RATE)

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

stop_on_val_loss = EarlyStopping(monitor='val_loss', min_delta=0.005, 
                                 patience=2, verbose=0, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, 
                              patience=2, min_lr=0.00001)

training_history = model.fit_generator(generator=make_generator("train"),
                                       steps_per_epoch=steps_per_epoch("train"),
                                       epochs=20,
                                       validation_data=make_generator("valid"),
                                       validation_steps=steps_per_epoch("valid"),
                                       callbacks=[stop_on_val_loss])

In [ ]:
model.save(os.path.join(DATADIR, datetime.datetime.today().strftime('%Y-%m-%d-%H-%M') + "-FIXED-DATA.h5"))

In [ ]:
def make_test_generator(folder="test",
                        data_flow_args={"seed": 1,
                                        "batch_size": BATCH_SIZE}):

    return ImageDataGenerator().flow_from_directory(
        directory=os.path.join(DATADIR, folder, "images"),
        target_size=(IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE),
        color_mode='rgb',
        **data_flow_args)

In [ ]:
test_batches = make_test_generator()
class_dict = test_batches.class_indices
index_dict = {i: c for c, i in class_dict.items()}

In [ ]:
def plot_confusion_matrix(matrix, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matrix[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
images, vectors = next(test_batches)
true_indices = vectors.argmax(1)
predictions = model.predict_on_batch(images)
prediction_indices = predictions.argmax(1)
class_names = [c for i, c in sorted(index_dict.items())]

cnf_matrix = confusion_matrix(true_indices, prediction_indices)
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')